In [ ]:
# Copyright 2025 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI Model Garden - Stable Diffusion XL 1.0 - TPU v5e

<table><tbody><tr>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/notebooks/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/community/model_garden/model_garden_jax_stable_diffusion_xl.ipynb">
      <img alt="Workbench logo" src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" width="32px"><br> Run in Workbench
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fvertex-ai-samples%2Fmain%2Fnotebooks%2Fcommunity%2Fmodel_garden%2Fmodel_garden_jax_stable_diffusion_xl.ipynb">
      <img alt="Google Cloud Colab Enterprise logo" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" width="32px"><br> Run in Colab Enterprise
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_garden/model_garden_jax_stable_diffusion_xl.ipynb">
      <img alt="GitHub logo" src="https://github.githubassets.com/assets/GitHub-Mark-ea2971cee799.png" width="32px"><br> View on GitHub
    </a>
  </td>
</tr></tbody></table>

## Overview

This notebook demonstrates how to deploy a **Stable-Diffusion-Xl-Base** open model on Google Cloud Vertex AI.

### Objectives

- Deploy Stable-Diffusion-Xl-Base using containerized backends like [vLLM](https://github.com/vllm-project/vllm) on GPU.
- Use the deployed model to serve chat completion requests for both text and multimodal inputs.

### File a Bug

If you encounter issues with this notebook, report them on [GitHub](https://github.com/GoogleCloudPlatform/vertex-ai-samples/issues/new).

### Costs

This tutorial uses billable components of Google Cloud:

- Vertex AI
- Cloud Storage

Refer to the [Vertex AI pricing](https://cloud.google.com/vertex-ai/pricing) and [Cloud Storage pricing](https://cloud.google.com/storage/pricing) pages for more information. Use the [Pricing Calculator](https://cloud.google.com/products/calculator/) to estimate your projected costs.

## Get Started

### Install Vertex AI SDK and other required packages

In [1]:
%pip install --upgrade --force-reinstall --quiet 'google-cloud-aiplatform>=1.106.0' 'openai' 'google-auth==2.27.0' 'requests==2.32.3'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.3/45.3 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.3/85.3 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 186.8/186.8 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 77.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 999.8/999.8 kB 46.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.1/109.1 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.3/150.3 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

### Authenticate the Notebook Environment (Colab only)

If you're running this notebook in Google Colab, run the following cell to authenticate.

In [2]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

### Set Google Cloud Project Information

To get started with Vertex AI, ensure you have an existing Google Cloud project and that the [Vertex AI API is enabled](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com).

See the guide on [setting up your project and development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment). Also confirm that [billing is enabled](https://cloud.google.com/billing/docs/how-to/modify-project).


In [4]:
# Use the environment variable if the user doesn't provide Project ID.
import os

import vertexai

PROJECT_ID = "qwiklabs-asl-01-752ad6b30635"  # @param {type: "string", placeholder: "[your-project-id]", isTemplate: true}

if not PROJECT_ID:
    PROJECT_ID = os.environ.get("GOOGLE_CLOUD_PROJECT")

REGION = "us-central1"  # @param {type: "string", placeholder: "[your-region]", isTemplate: true}

if not REGION:
    REGION = os.environ.get("GOOGLE_CLOUD_REGION", "us-central1")

vertexai.init(project=PROJECT_ID, location=REGION)

print(f"Project: {PROJECT_ID}\nLocation: {REGION}")

Project: qwiklabs-asl-01-752ad6b30635
Location: us-central1


In [ ]:
# ValueError: Unsupported region for Vertex AI, select from frozenset({'us-west1', 'us-south1', 'europe-west2', 'asia-northeast1', 'southamerica-west1', 'asia-south2', 'asia-southeast1', 'us-west2', 'australia-southeast2', 'asia-east2', 'global', 'me-west1', 'europe-west1', 'europe-west8', 'asia-southeast2', 'northamerica-northeast2', 'us-east5', 'us-east7', 'southamerica-east1', 'europe-west6', 'me-central1', 'asia-south1', 'europe-west4', 'us-central1', 'us-west4', 'asia-northeast3', 'asia-east1', 'us-east4', 'australia-southeast1', 'us-west3', 'me-central2', 'us-east1', 'africa-south1', 'asia-northeast2', 'northamerica-northeast1', 'europe-southwest1', 'europe-north1', 'europe-west9', 'europe-west3', 'europe-central2', 'europe-west12'})


### Import libraries

In [5]:
from vertexai import model_garden

## Deploy model

### Choose model variant

You can proceed with the default model variant or select a different one.

In [6]:
model_version = "stable-diffusion-xl-base-1.0"  # @param ["stable-diffusion-xl-base-1.0"] {isTemplate:true}
MODEL_NAME = f"stability-ai/stable-diffusion-xl-base@{model_version}"

To see all deployable model variants available in Model Garden, use:

In [7]:
all_model_versions = model_garden.list_deployable_models(
    model_filter="stable-diffusion-xl-base", list_hf_models=False
)

Once you've selected a model variant, initialize it:

In [8]:
model = model_garden.OpenModel(MODEL_NAME)

In [9]:
model

### Check the Deployment Configuration

Use the `list_deploy_options()` method to view the verified deployment configurations for your selected model. This helps ensure you have sufficient resources (e.g., GPU quota) available to deploy it.

In [10]:
deploy_options = model.list_deploy_options(concise=True)
print(deploy_options)

[Option 1: 1 NVIDIA_A100_80GB a2-ultragpu-1g]
    serving_container_image_uri="us-docker.pkg.dev/deeplearning-platform-release/vertex-model-garden/pytorch-inference.cu125.0-4.ubuntu2204.py310",
    machine_type="a2-ultragpu-1g",
    accelerator_type="NVIDIA_A100_80GB",
    accelerator_count=1,

[Option 2: 1 NVIDIA_L4 g2-standard-8]
    serving_container_image_uri="us-docker.pkg.dev/deeplearning-platform-release/vertex-model-garden/pytorch-inference.cu125.0-4.ubuntu2204.py310",
    machine_type="g2-standard-8",
    accelerator_type="NVIDIA_L4",
    accelerator_count=1,

[Option 3: 1 NVIDIA_TESLA_T4 n1-standard-4]
    serving_container_image_uri="us-docker.pkg.dev/deeplearning-platform-release/vertex-model-garden/pytorch-inference.cu125.0-4.ubuntu2204.py310",
    machine_type="n1-standard-4",
    accelerator_type="NVIDIA_TESLA_T4",
    accelerator_count=1,

[Option 4: 1 NVIDIA_TESLA_V100 n1-standard-4]
    serving_container_image_uri="us-docker.pkg.dev/deeplearning-platform-release/verte

In [ ]:
# What kind of machine will be used by the model endpoint?
# NOT n1-standard-4
# NOT a3-highgpu-1g
# NOT n1-highcpu-64
# ct5lp-hightpu-1t -> WEIRD - THIS DOES NOT APPEAR ABOVE OR BELOW!



### Deploy the Model

Now that you’ve reviewed the deployment options, use the `deploy()` method to serve the selected open model to a Vertex AI endpoint. Deployment time may vary depending on the model size and infrastructure requirements.

> **Note**: If the model requires accepting a license agreement (EULA), set the `accept_eula=True` flag in the deploy call. Set `use_dedicated_endpoint` to False if you don't want to use [dedicated endpoint](https://cloud.google.com/vertex-ai/docs/general/deployment#create-dedicated-endpoint).

In [ ]:
use_dedicated_endpoint = True

In [ ]:
endpoints = {}

In [ ]:
endpoints["sdk_default"] = model.deploy(
    accept_eula=True,
    use_dedicated_endpoint=use_dedicated_endpoint,
)

Alternatively, you can select one of the verified deployment configurations listed above.

In [ ]:
endpoints["sdk_custom"] = model.deploy(
    accept_eula=True,
    use_dedicated_endpoint=use_dedicated_endpoint,
    serving_container_image_uri="us-docker.pkg.dev/deeplearning-platform-release/vertex-model-garden/pytorch-inference.cu125.0-4.ubuntu2204.py310",
    machine_type="a2-ultragpu-1g",
    accelerator_type="NVIDIA_A100_80GB",
    accelerator_count=1,
)

In [ ]:
if "sdk_default" in endpoints:
    endpoint = endpoints["sdk_default"]
elif "sdk_custom" in endpoints:
    endpoint = endpoints["sdk_custom"]
else:
    raise ValueError("No endpoint found. Create an endpoint.")

To further customize your deployment, you can configure:

- **Compute Resources**: Machine type, replica count (min/max), accelerator type and quantity.
- **Infrastructure**: Use Spot VMs, reservation affinity, or dedicated endpoints.
- **Serving Container**: Customize container image, ports, health checks, and environment variables.

See the [Model Garden SDK README](https://github.com/googleapis/python-aiplatform/blob/main/vertexai/model_garden/README.md) for advanced configuration options.

In [ ]:
# @title Predict

# @markdown Once deployed, you can send a batch of text prompts to the endpoint to generated images.

# @markdown When deployed on one TPU V5e instance, the averaged inference time of one image is ~3 seconds.


import base64
from io import BytesIO

from PIL import Image


def base64_to_image(image_str):
    """Convert base64 encoded string to an image."""
    image = Image.open(BytesIO(base64.b64decode(image_str)))
    return image


def image_grid(imgs, rows=2, cols=2):
    w, h = imgs[0].size
    grid = Image.new(
        mode="RGB", size=(cols * w + 10 * cols, rows * h), color=(255, 255, 255)
    )
    for i, img in enumerate(imgs):
        grid.paste(img, box=(i % cols * w + 10 * i, i // cols * h))
    return grid


instances = [
    {
        "prompt": "Photorealistic whale swimming in abyss",
        "height": 1024,
        "width": 1024,
    },
]
response = endpoint.predict(instances=instances)

images = [
    base64_to_image(response.predictions[0]) for prediction in response.predictions
]

image_grid(images, rows=1)

### Clean up resources

In [ ]:
# @markdown Delete the endpoint.

for endpoint in endpoints.values():
    endpoint.delete(force=True)